In [1]:
!pip install tensorflow
!pip install tfrecord


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.9/52.9 kB 2.0 MB/s eta 0:00:00
  Created wheel for tfrecord: filename=tfrecord-1.14.5-py3-none-any.whl size=14907 sha256=f56ac9f4644ba15e2af7eb6617aa085b7309199b75072636b934e5d8b1c299fd
  Stored in directory: /root/.cache/pip/wheels/1d/c1/9d/7a575d075fde1b0c5e910bd3baffd13e8dee088323f0f07797
Successfully built tfrecord


In [2]:
!rm -f yt8m_data/*.tfrecord


In [2]:
import os

# Tạo thư mục nếu chưa tồn tại
os.makedirs('yt8m_data', exist_ok=True)

# Số lượng file muốn tải về
num_files = 3844  # Thay đổi giá trị này nếu cần

# Định nghĩa các ký tự cho x và y
chars_x = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']  # Thêm các ký tự khác nếu cần
chars_y = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']  # Thêm các ký tự khác nếu cần

# Tạo danh sách file_paths
file_paths = [f"yt8m_data/train{cx}{cy}.tfrecord" for cx in chars_x for cy in chars_y]

# Giới hạn số lượng file theo num_files
file_paths = file_paths[:num_files]

# Tải về các file (nếu cần)
for fp in file_paths:
    url = f'http://us.data.yt8m.org/2/video/train/{os.path.basename(fp)}'
    os.system(f'wget -P yt8m_data/ {url}')

# Tạo dataset cho từng file
# datasets = [create_dataset(fp) for fp in file_paths]



In [3]:
import tensorflow as tf

# Định nghĩa hàm giải mã các mẫu từ tfrecord
def parse_tfrecord_fn(example):
    feature_description = {
        'id': tf.io.FixedLenFeature([], tf.string),
        'labels': tf.io.VarLenFeature(tf.int64),
        'mean_audio': tf.io.FixedLenFeature([128], tf.float32),  # Mean audio với 128 chiều
    }
    parsed_example = tf.io.parse_single_example(example, feature_description)
    parsed_example['labels'] = tf.sparse.to_dense(parsed_example['labels'])  # Chuyển labels từ sparse sang dense
    return parsed_example

# Hàm để tạo dataset từ file .tfrecord
def create_dataset(file_path):
    raw_dataset = tf.data.TFRecordDataset(file_path)
    dataset = raw_dataset.map(parse_tfrecord_fn)
    return dataset

# Số lượng file muốn sử dụng
num_files = 3844  # Thay đổi giá trị này nếu cần

# Định nghĩa các ký tự cho x và y
chars_x = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']  # Ký tự cho x
chars_y = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']  # Ký tự cho y

# Tạo danh sách file_paths
file_paths = []
for cx in chars_x:
    for cy in chars_y:
        file_path = f"yt8m_data/train{cx}{cy}.tfrecord"
        file_paths.append(file_path)
        if len(file_paths) >= num_files:  # Giới hạn số lượng file
            break
    if len(file_paths) >= num_files:
        break

# Tạo dataset cho từng file
datasets = [create_dataset(fp) for fp in file_paths]


# Kết hợp tất cả các datasets thành một
full_dataset = datasets[0]
for ds in datasets[1:]:
    full_dataset = full_dataset.concatenate(ds)

# Lấy một số mẫu từ dataset để kiểm tra
for data in full_dataset.take(1):
    print(data)


{'labels': <tf.Tensor: shape=(6,), dtype=int64, numpy=array([  0,   1,   5,  69, 378, 597])>, 'id': <tf.Tensor: shape=(), dtype=string, numpy=b'mpaa'>, 'mean_audio': <tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.2909387 ,  0.2640953 , -1.3249685 ,  0.18124525, -0.3696523 ,
       -1.4134524 , -0.1309337 , -0.99610907,  0.2916015 , -0.7138666 ,
       -1.0553192 , -0.13877684, -1.0889012 , -0.02831012,  0.6336065 ,
        0.50789535,  0.91352916,  0.21107127, -0.02499612, -0.28039518,
       -0.51347995,  0.85487133, -0.66835433,  0.26508948, -0.01682158,
        0.31932867,  0.0722146 ,  0.0814938 , -0.77230346,  1.2726564 ,
       -0.01925185,  0.16412291, -0.5120439 ,  0.22819361,  0.67690945,
       -0.21058021,  0.06481333, -0.888404  ,  0.04879566, -0.16473652,
        0.0114579 , -0.22063269, -0.18053326,  0.08237754, -0.7503206 ,
        0.46934247, -0.9624167 ,  0.8739821 ,  0.3453988 ,  0.37489343,
        0.6412287 ,  0.7562245 ,  0.08160427, -0.66923803,  0.5306

In [4]:
import numpy as np

# Hàm để tìm số lượng nhãn duy nhất (num_classes) trong dataset
def get_num_classes(dataset):
    unique_labels = set()  # Sử dụng set để lưu trữ các nhãn duy nhất
    for data in dataset:
        labels = data['labels'].numpy()  # Lấy nhãn từ bản ghi
        unique_labels.update(labels)  # Cập nhật tập hợp các nhãn duy nhất
    print(len(unique_labels))
    return len(unique_labels)  # Trả về số lượng nhãn duy nhất

# Hàm để trích xuất X và y từ dataset
def get_X_y_from_dataset(dataset):
    num_classes = get_num_classes(dataset)  # Tính số lượng lớp
    X = []
    y = []
    for data in dataset:
        X.append(data['mean_audio'].numpy())  # Lấy dữ liệu từ mean_audio
        labels = data['labels'].numpy()  # Lấy nhãn từ trường labels

        # Kiểm tra và đảm bảo không có nhãn nào vượt quá số lượng lớp
        valid_labels = labels[labels < num_classes]  # Lọc nhãn hợp lệ
        one_hot_labels = np.zeros(num_classes)  # Giả sử có lớp
        one_hot_labels[valid_labels] = 1  # Đánh dấu các nhãn hợp lệ
        y.append(one_hot_labels)

    return np.array(X), np.array(y)

# Chuyển đổi thành X_train và y_train
X_train, y_train = get_X_y_from_dataset(full_dataset)

# Kiểm tra kích thước
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)


3862
X_train shape: (3888919, 128)
y_train shape: (3888919, 3862)


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Xây dựng mô hình
model = Sequential([
    Dense(512, activation='relu', input_shape=(128,)),  # mean_audio có kích thước 128
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(3862, activation='sigmoid')  # Sử dụng sigmoid cho multi-label classification
])

# Compile mô hình với loss function phù hợp
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Hiển thị cấu trúc mô hình
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               66048     
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 3862)              498198    
                                                                 
Total params: 728470 (2.78 MB)
Trainable params: 728470 (2.78 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

# Giả sử bạn đã có X_train và y_train
num_classes = get_num_classes(full_dataset)  # Tính số lớp

# Tạo mô hình
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))  # Đầu vào
model.add(Dense(num_classes, activation='sigmoid'))  # Đầu ra

# Biên dịch mô hình
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(X_train, y_train, epochs=10, batch_size=32)


In [31]:


# Lưu mô hình đã huấn luyện
model.save('video_recommendation_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [32]:
# Dự đoán nhãn cho dữ liệu mới
predictions = model.predict(X_train[:5])  # Dự đoán cho 5 mẫu đầu tiên
print("Predictions: ", predictions)

# Đánh giá mô hình
loss, accuracy = model.evaluate(X_train, y_train)
print(f"Loss: {loss}, Accuracy: {accuracy}")


1/1 [==============================] - 0s 72ms/step
Predictions:  [[8.7235463e-01 8.2029647e-01 2.9386684e-02 ... 1.7960519e-04
  2.9695936e-04 1.6284011e-04]
 [8.1782532e-01 8.7422264e-01 1.4074171e-02 ... 1.8082128e-05
  5.7558871e-05 3.9397852e-05]
 [2.7415074e-02 1.1657115e-02 4.0635854e-02 ... 2.8127439e-05
  4.0911742e-05 3.9143568e-05]
 [7.7827625e-02 4.5229089e-02 2.0436260e-01 ... 1.7327612e-04
  3.6897938e-04 2.8344768e-04]
 [1.9889239e-03 5.0537038e-04 6.0656253e-02 ... 6.1273146e-05
  2.7031227e-05 2.6381054e-05]]
315/315 [==============================] - 1s 2ms/step - loss: 0.0040 - accuracy: 0.3978
Loss: 0.004015731159597635, Accuracy: 0.3977510333061218
